In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 1: Import Libraries

In [2]:
# Import Libraries

import time
import pickle
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from sklearn.svm import SVC
from sklearn.metrics import auc
from collections import Counter
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn import preprocessing
from prettytable import PrettyTable 
from sklearn.metrics import roc_curve
from astropy.table import Table, Column
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection

##### kfold cross validation ################
from sklearn.model_selection import RepeatedStratifiedKFold

###### SMOTE and RandomUnderSampler (for balancing data) ############

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import SVMSMOTE 
from imblearn.over_sampling import ADASYN 
from imblearn.under_sampling import NearMiss 

########### unsupervised learning for outliers ########################
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN

########## #########
from numpy import hstack
from numpy import array

# Step 2: Read Dataset

In [3]:
# Load Dataset

train = pd.read_csv('drive/My Drive/Colab Notebooks/WisconsinDataset/stackingPlus/work/train.csv') 
test = pd.read_csv('drive/My Drive/Colab Notebooks/WisconsinDataset/stackingPlus/work/test.csv') 


In [4]:
# Understand Dataset
print("train set:")
print("===================")
print(train['Flagged'].value_counts())

print("\ntest set:")
print("===================")
print(test['Flagged'].value_counts())

train set:
0    1277
1      31
Name: Flagged, dtype: int64

test set:
0    313
1     15
Name: Flagged, dtype: int64


# Step 4: Pre-Process Dataset

In [5]:
# Pre-Processing
train.drop(train.loc[:, 'iresp.1':'iresp.170'], inplace = True, axis = 1) 
test.drop(test.loc[:, 'iresp.1':'iresp.170'], inplace = True, axis = 1) 

In [6]:
train['mean_idur'] = train.loc[:, 'idur.1':'idur.170'].mean(axis=1)
train['median_idur'] = train.loc[:, 'idur.1':'idur.170'].median(axis=1)
train['max_idur'] = train.loc[:, 'idur.1':'idur.170'].max(axis=1)
train['min_idur'] = train.loc[:, 'idur.1':'idur.170'].min(axis=1)

# Correct Answers

train['iraw_score'] = train.loc[:, 'iraw.1':'iraw.170'].sum(axis=1)
train['iraw_score_pct'] = train['iraw_score']/170

In [7]:
test['mean_idur'] = test.loc[:, 'idur.1':'idur.170'].mean(axis=1)
test['median_idur'] = test.loc[:, 'idur.1':'idur.170'].median(axis=1)
test['max_idur'] = test.loc[:, 'idur.1':'idur.170'].max(axis=1)
test['min_idur'] = test.loc[:, 'idur.1':'idur.170'].min(axis=1)

# Correct Answers

test['iraw_score'] = test.loc[:, 'iraw.1':'iraw.170'].sum(axis=1)
test['iraw_score_pct'] = test['iraw_score']/170

# Step 5: Feature Selection

In [8]:
# iraw_idur_OEF (both+OEF)

col_features = ['Attempt', 'tot_time', 'mean_idur', 'median_idur', 'max_idur', 'min_idur', 'iraw_score_pct']

for x in range(1,171):
    col_features.append('idur.'+str(x))

for i in range(1,171):
    col_features.append('iraw.'+str(i))

X_train = train.loc[:, col_features]
y_train = train.Flagged    

X_test = test.loc[:, col_features]
y_test = test.Flagged    


In [9]:
# Count Values

counter_train = Counter(y_train)
print(counter_train)

counter_test = Counter(y_test)
print(counter_test)

Counter({0: 1277, 1: 31})
Counter({0: 313, 1: 15})


# Step 7: Feature Scaling

In [10]:
####### scale the training set in array due to kfold needed ############
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)


In [11]:
# Import Libraries
# from sklearn.preprocessing import MinMaxScaler
# num_cols = X_train.columns

# # apply standardization on numerical features
# for i in num_cols:
#     # fit on training data column
#     scale = MinMaxScaler(feature_range=(0, 1)).fit(X_train[[i]])

#     # transform the training data column
#     X_train[i] = scale.transform(X_train[[i]])


In [12]:
######## scale the test set ################
from sklearn.preprocessing import MinMaxScaler
num_cols = X_test.columns

# apply standardization on numerical features
for i in num_cols:
    # fit on training data column
    scale = MinMaxScaler(feature_range=(0, 1)).fit(X_test[[i]])

    # transform the training data column
    X_test[i] = scale.transform(X_test[[i]])


<center>
    <br>
        <br>
            <br>
                <h5 style = "color:blue">====================================================================</h5>
                <h2 style = "color:green">Constructing Meta Model</h2>
                <h5 style = "color:blue">====================================================================</h5>
            <br>
        <br>
    <br>
</center>

In [13]:
##### resampling ##########
### both 33%
#over = SMOTE(sampling_strategy=0.4)
over = SVMSMOTE(sampling_strategy=0.4, random_state=42, k_neighbors=10)
#over = ADASYN(sampling_strategy=0.4, random_state=42)  ###bad

#under = RandomUnderSampler(sampling_strategy=0.5)
under = NearMiss(sampling_strategy=0.5)

steps = [('o', over), ('u', under)]

pipeline = Pipeline(steps=steps)


#X_train, X_test, y_train, y_test= train_test_split(X, Y, test_size= 0.2, random_state=23 ,shuffle = True)  
#### X_train1_orig, X_pred_train, y_train1_orig, y_pred_train= train_test_split(X_train, y_train, test_size= 0.25, random_state=23 ,shuffle = True)  ### remove this line and change to kfold
#### X_train1, y_train1 = pipeline.fit_resample(X_train1_orig, y_train1_orig)   ### move this line into kfold loop


In [14]:
from xlwt import Workbook 
wb = Workbook() 

In [15]:
X_train.shape

(1308, 347)

In [16]:
k_fold = model_selection.KFold(n_splits = 4, random_state=25, shuffle=True)

full_pretrainset_x, full_pretrainset_y = list(), list()
full_pred1, full_pred2, full_pred3, full_pred4 = list(), list(), list(), list()
full_outlier1, full_outlier2, full_outlier3, full_outlier4, full_outlier5 = list(), list(), list(), list(), list()

i = 0
for train_ix, test_ix in k_fold.split(X_train):
    i = i + 1

    # kfold
    X_train1_orig, y_train1_orig, X_pred_train, y_pred_train = X_train[train_ix], y_train[train_ix], X_train[test_ix], y_train[test_ix]

    X_train1, y_train1 = pipeline.fit_resample(X_train1_orig, y_train1_orig)  ### apply resampling on training set

    # Training
    clf1 = RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
    clf1.fit(X_train1, y_train1)

    clf2 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100)
    clf2.fit(X_train1, y_train1)

    clf3 = MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam")
    clf3.fit(X_train1, y_train1)

    clf4 = GaussianNB(var_smoothing=6.579332246575682e-08)
    clf4.fit(X_train1, y_train1)

    # ############ 4 predictions on pre_train set ##############
    # P1 = clf1.predict_proba(X_pred_train)
    # P1 = P1[:, 1]
    # P2 = clf2.predict_proba(X_pred_train)
    # P2 = P2[:, 1]
    # P3 = clf3.predict_proba(X_pred_train)
    # P3 = P3[:, 1]
    # P4 = clf4.predict_proba(X_pred_train)
    # P4 = P4[:, 1]

    P1 = clf1.predict_proba(X_pred_train)[:, 0]
    full_pred1.extend(P1)

    P2 = clf2.predict_proba(X_pred_train)[:, 0]
    full_pred2.extend(P2)

    P3 = clf3.predict_proba(X_pred_train)[:, 0]
    full_pred3.extend(P3)

    P4 = clf4.predict_proba(X_pred_train)[:, 0]
    full_pred4.extend(P4)

    ############ 5 outliers on pred_train set ##############

    Isolation_Forest = IsolationForest(random_state=64)
    outlier1 = Isolation_Forest.fit_predict(X_pred_train)
    # print("Number Outliers Detected: ", list(outlier1).count(-1))
    # print("Other 1: ", list(outlier1).count(1))
    full_outlier1.extend(outlier1)

    Elliptic_Envelope = EllipticEnvelope()
    outlier2 = Elliptic_Envelope.fit_predict(X_pred_train)
    # print("Number Outliers Detected: ", list(outlier2).count(-1))
    # print("Other 1: ", list(outlier2).count(1))
    full_outlier2.extend(outlier2)

    Local_Outlier_Factor = LocalOutlierFactor()
    outlier3 = Local_Outlier_Factor.fit_predict(X_pred_train)
    # print("Number Outliers Detected: ", list(outlier3).count(-1))
    # print("Other 1: ", list(outlier3).count(1))
    full_outlier3.extend(outlier3)

    One_Class_SVM = OneClassSVM(kernel='linear', tol=0.001, nu=0.05)
    outlier4 = One_Class_SVM.fit_predict(X_pred_train)
    # print("Number Outliers Detected: ", list(outlier4).count(-1))
    # print("Other 1: ", list(outlier4).count(1))
    full_outlier4.extend(outlier4)

    outlier_detection = DBSCAN(min_samples = 300, eps = 8.2)
    outlier5 = outlier_detection.fit_predict(X_pred_train)
    # print("Number Outliers Detected: ", list(outlier5).count(-1))
    # print("Other 0: ", list(outlier5).count(0))
    full_outlier5.extend(outlier5)

    ####### construct meta model's training dataset ############
    full_pretrainset_x.extend(X_pred_train) 
    full_pretrainset_y.extend(y_pred_train)


In [17]:
pred1 = array(full_pred1).reshape((len(full_pred1), 1))
pred2 = array(full_pred2).reshape((len(full_pred2), 1))
pred3 = array(full_pred3).reshape((len(full_pred3), 1))
pred4 = array(full_pred4).reshape((len(full_pred4), 1))

outlr1 = array(full_outlier1).reshape((len(full_outlier1), 1))
outlr2 = array(full_outlier2).reshape((len(full_outlier2), 1))
outlr3 = array(full_outlier3).reshape((len(full_outlier3), 1))
outlr4 = array(full_outlier4).reshape((len(full_outlier4), 1))
outlr5 = array(full_outlier5).reshape((len(full_outlier5), 1))

# stack as separate columns
full_pretrainset_x = hstack((full_pretrainset_x, pred1, pred2, pred3, pred4, outlr1, outlr2, outlr3, outlr4, outlr5))

In [18]:
col_features.append('Predict1')
col_features.append('Predict2')
col_features.append('Predict3')
col_features.append('Predict4')
col_features.append('Outlier1')
col_features.append('Outlier2')
col_features.append('Outlier3')
col_features.append('Outlier4')
col_features.append('Outlier5')

print(col_features)

['Attempt', 'tot_time', 'mean_idur', 'median_idur', 'max_idur', 'min_idur', 'iraw_score_pct', 'idur.1', 'idur.2', 'idur.3', 'idur.4', 'idur.5', 'idur.6', 'idur.7', 'idur.8', 'idur.9', 'idur.10', 'idur.11', 'idur.12', 'idur.13', 'idur.14', 'idur.15', 'idur.16', 'idur.17', 'idur.18', 'idur.19', 'idur.20', 'idur.21', 'idur.22', 'idur.23', 'idur.24', 'idur.25', 'idur.26', 'idur.27', 'idur.28', 'idur.29', 'idur.30', 'idur.31', 'idur.32', 'idur.33', 'idur.34', 'idur.35', 'idur.36', 'idur.37', 'idur.38', 'idur.39', 'idur.40', 'idur.41', 'idur.42', 'idur.43', 'idur.44', 'idur.45', 'idur.46', 'idur.47', 'idur.48', 'idur.49', 'idur.50', 'idur.51', 'idur.52', 'idur.53', 'idur.54', 'idur.55', 'idur.56', 'idur.57', 'idur.58', 'idur.59', 'idur.60', 'idur.61', 'idur.62', 'idur.63', 'idur.64', 'idur.65', 'idur.66', 'idur.67', 'idur.68', 'idur.69', 'idur.70', 'idur.71', 'idur.72', 'idur.73', 'idur.74', 'idur.75', 'idur.76', 'idur.77', 'idur.78', 'idur.79', 'idur.80', 'idur.81', 'idur.82', 'idur.83', 'i

In [19]:
meta_x = pd.DataFrame(full_pretrainset_x, columns=col_features)

In [20]:
meta_x

,Attempt,tot_time,mean_idur,median_idur,max_idur,min_idur,iraw_score_pct,idur.1,idur.2,idur.3,...,iraw.170,Predict1,Predict2,Predict3,Predict4,Outlier1,Outlier2,Outlier3,Outlier4,Outlier5
0,0.2,0.303289,0.313774,0.235897,0.110314,0.206897,0.813725,0.089928,0.074913,0.107492,...,0.0,0.928650,0.999504,0.999614,1.000000e+00,1.0,1.0,1.0,1.0,-1.0
1,0.6,0.392403,0.399302,0.394872,0.071749,0.103448,0.215686,0.071942,0.069686,0.364821,...,1.0,0.692645,0.994653,0.999912,1.830347e-50,-1.0,1.0,1.0,-1.0,-1.0
2,0.8,0.390418,0.394378,0.312821,0.140807,0.482759,0.401961,0.348921,0.043554,0.130293,...,1.0,0.591658,0.956908,0.175505,1.337760e-12,-1.0,1.0,1.0,1.0,-1.0
3,0.2,0.396212,0.402393,0.364103,0.130045,0.551724,0.794118,0.064748,0.146341,0.540717,...,1.0,0.967069,0.999345,0.999804,1.000000e+00,1.0,1.0,1.0,1.0,-1.0
4,0.2,0.228768,0.231452,0.174359,0.085202,0.413793,0.607843,0.205036,0.134146,0.058632,...,0.0,0.868277,0.983572,0.999707,1.240882e-05,1.0,1.0,1.0,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,0.2,0.363324,0.374628,0.364103,0.096861,0.413793,0.372549,0.302158,0.085366,0.182410,...,0.0,0.716795,0.749793,0.999995,1.000000e+00,1.0,1.0,1.0,-1.0,-1.0
1304,0.2,0.270615,0.276391,0.220513,0.165022,0.172414,0.803922,0.140288,0.297909,0.078176,...,1.0,0.911087,0.991671,0.999909,1.000000e+00,1.0,1.0,1.0,1.0,-1.0
1305,0.2,0.393691,0.403423,0.410256,0.124664,0.310345,0.549020,0.107914,0.055749,0.332248,...,1.0,0.840834,0.993173,0.964829,1.000000e+00,1.0,1.0,1.0,1.0,-1.0
1306,1.0,0.335587,0.344516,0.297436,0.152466,0.379310,0.274510,0.151079,0.113240,0.221498,...,1.0,0.660812,0.889808,0.497246,6.586491e-54,-1.0,1.0,1.0,-1.0,-1.0


In [21]:

# summatives + top features from 4 selection algorithm + prediction1-4
col_features = ['Attempt', 'tot_time', 'mean_idur', 'median_idur', 'max_idur', 'min_idur', 'iraw_score_pct', 'iraw.32', 'iraw.77', 'iraw.147', 'iraw.57', 'iraw.86', 'iraw.152', 'iraw.12', 'iraw.127', 'iraw.140', 'iraw.96', 'idur.146', 'idur.36', 'idur.114', 'idur.84', 'idur.142', 'idur.60', 'idur.71', 'idur.133', 'idur.88', 'idur.59', 'idur.69', 'idur.38', 'idur.67', 'idur.81', 'idur.96', 'idur.162', 'Predict2', 'Predict3', 'Predict4', 'Outlier1', 'Outlier2', 'Outlier3', 'Outlier4', 'Outlier5']
X_pred_train_meta = meta_x.loc[:, col_features]


In [22]:
X_pred_train_meta

,Attempt,tot_time,mean_idur,median_idur,max_idur,min_idur,iraw_score_pct,iraw.32,iraw.77,iraw.147,...,idur.96,idur.162,Predict2,Predict3,Predict4,Outlier1,Outlier2,Outlier3,Outlier4,Outlier5
0,0.2,0.303289,0.313774,0.235897,0.110314,0.206897,0.813725,1.0,1.0,1.0,...,0.065,0.249284,0.999504,0.999614,1.000000e+00,1.0,1.0,1.0,1.0,-1.0
1,0.6,0.392403,0.399302,0.394872,0.071749,0.103448,0.215686,0.0,0.0,1.0,...,0.260,0.180516,0.994653,0.999912,1.830347e-50,-1.0,1.0,1.0,-1.0,-1.0
2,0.8,0.390418,0.394378,0.312821,0.140807,0.482759,0.401961,0.0,0.0,0.0,...,0.420,0.283668,0.956908,0.175505,1.337760e-12,-1.0,1.0,1.0,1.0,-1.0
3,0.2,0.396212,0.402393,0.364103,0.130045,0.551724,0.794118,0.0,0.0,0.0,...,0.130,0.272206,0.999345,0.999804,1.000000e+00,1.0,1.0,1.0,1.0,-1.0
4,0.2,0.228768,0.231452,0.174359,0.085202,0.413793,0.607843,1.0,1.0,0.0,...,0.250,0.126074,0.983572,0.999707,1.240882e-05,1.0,1.0,1.0,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,0.2,0.363324,0.374628,0.364103,0.096861,0.413793,0.372549,0.0,0.0,0.0,...,0.235,0.358166,0.749793,0.999995,1.000000e+00,1.0,1.0,1.0,-1.0,-1.0
1304,0.2,0.270615,0.276391,0.220513,0.165022,0.172414,0.803922,0.0,0.0,0.0,...,0.165,0.106017,0.991671,0.999909,1.000000e+00,1.0,1.0,1.0,1.0,-1.0
1305,0.2,0.393691,0.403423,0.410256,0.124664,0.310345,0.549020,0.0,0.0,0.0,...,0.225,0.283668,0.993173,0.964829,1.000000e+00,1.0,1.0,1.0,1.0,-1.0
1306,1.0,0.335587,0.344516,0.297436,0.152466,0.379310,0.274510,0.0,0.0,1.0,...,0.455,0.252149,0.889808,0.497246,6.586491e-54,-1.0,1.0,1.0,-1.0,-1.0


In [23]:
combined_model = GradientBoostingClassifier()
#combined_model = RandomForestClassifier()
#combined_model.fit(X_pred_train_meta, y_pred_train)
combined_model.fit(X_pred_train_meta, full_pretrainset_y)


GradientBoostingClassifier()

## Test set

In [24]:
# scores on test data
###
y_pred = clf1.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- RF ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

###
y_pred = clf2.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- GB ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

###
y_pred = clf3.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- NN ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

###
y_pred = clf2.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- NB ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

  -------------------- RF ------------------------- 
  precision:  1.0
  recall:  0.06666666666666667
  f1-score:  0.125
  fpr:  0.0
  tn:  313
  fp:  0
  fn:  14
  tp:  1
  real_cheaters:  15
  -------------------- GB ------------------------- 
  precision:  0.47368421052631576
  recall:  0.6
  f1-score:  0.5294117647058824
  fpr:  0.03194888178913738
  tn:  303
  fp:  10
  fn:  6
  tp:  9
  real_cheaters:  15
  -------------------- NN ------------------------- 
  precision:  0.6
  recall:  0.2
  f1-score:  0.3
  fpr:  0.006389776357827476
  tn:  311
  fp:  2
  fn:  12
  tp:  3
  real_cheaters:  15
  -------------------- NB ------------------------- 
  precision:  0.47368421052631576
  recall:  0.6
  f1-score:  0.5294117647058824
  fpr:  0.03194888178913738
  tn:  303
  fp:  10
  fn:  6
  tp:  9
  real_cheaters:  15


In [25]:
############ 5 outliers on test set ##############

Isolation_Forest = IsolationForest(random_state=64)
outlier1_test = Isolation_Forest.fit_predict(X_test)
print("Number Outliers Detected: ", list(outlier1_test).count(-1))
print("Other 1: ", list(outlier1_test).count(1))

Elliptic_Envelope = EllipticEnvelope()
outlier2_test = Elliptic_Envelope.fit_predict(X_test)
print("Number Outliers Detected: ", list(outlier2_test).count(-1))
print("Other 1: ", list(outlier2_test).count(1))

Local_Outlier_Factor = LocalOutlierFactor()
outlier3_test = Local_Outlier_Factor.fit_predict(X_test)
print("Number Outliers Detected: ", list(outlier3_test).count(-1))
print("Other 1: ", list(outlier3_test).count(1))

One_Class_SVM = OneClassSVM(kernel='linear', tol=0.001, nu=0.05)
outlier4_test = One_Class_SVM.fit_predict(X_test)
print("Number Outliers Detected: ", list(outlier4_test).count(-1))
print("Other 1: ", list(outlier4_test).count(1))

outlier_detection = DBSCAN(min_samples = 300, eps = 8.5)
outlier5_test = outlier_detection.fit_predict(X_test)
print("Number Outliers Detected: ", list(outlier5_test).count(-1))
print("Other 0: ", list(outlier5_test).count(0))


Number Outliers Detected:  36
Other 1:  292
Number Outliers Detected:  36
Other 1:  292
Number Outliers Detected:  0
Other 1:  328
Number Outliers Detected:  18
Other 1:  310
Number Outliers Detected:  328
Other 0:  0


In [26]:
############ 4 predictions on test set ################

P1_test = clf1.predict_proba(X_test)
P1_test = P1_test[:, 1]
P2_test = clf2.predict_proba(X_test)
P2_test = P2_test[:, 1]
P3_test = clf3.predict_proba(X_test)
P3_test = P3_test[:, 1]
P4_test = clf4.predict_proba(X_test)
P4_test = P4_test[:, 1]

In [27]:
X_test["Predict1"] = P1_test
X_test["Predict2"] = P2_test
X_test["Predict3"] = P3_test
X_test["Predict4"] = P4_test

X_test["Outlier1"] = outlier1_test
X_test["Outlier2"] = outlier2_test
X_test["Outlier3"] = outlier3_test
X_test["Outlier4"] = outlier4_test
X_test["Outlier5"] = outlier5_test

X_test_meta = X_test.loc[:, col_features]   ### top features + prediction1~4 + outlier1~5

combined_model_predictions = combined_model.predict(X_test_meta)

In [28]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [29]:
tn, fp, fn, tp = confusion_matrix(y_test, combined_model_predictions).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_test, combined_model_predictions))
print("  recall: ", recall_score(y_test, combined_model_predictions))
print("  f1-score: ", f1_score(y_test, combined_model_predictions))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)


  precision:  0.04597701149425287
  recall:  0.8
  f1-score:  0.08695652173913043
  fpr:  0.7955271565495208
  tn:  64
  fp:  249
  fn:  3
  tp:  12
  real_cheaters:  15
